# gathering metadata on assistive technology

using github graphql api to gather metadata about
projects tagged with assistive technology.

this exploration looks specifically at assistive technology tags.
this might be a good _training_ set for a more open general search
that tries to differentiate between assistive tech presenting and the real deal.

In [1]:
    __import__("dotenv").load_dotenv()
    client = __import__("python_graphql_client").GraphqlClient(
        "https://api.github.com/graphql", dict(Authorization=F"token {os.environ['GITHUB_TOKEN']}")
    )


In [41]:
%%
    async def search_one(query, after=None):
<details><summary>the graphql query</summary>

with REST, you get what you get. with grapql, you get what you ask for.

        return await client.execute_async(
```graphql
{
      search(query: "%s", type: REPOSITORY, first: 50, after: %s) {
        repositoryCount
    wikiCount
    pageInfo {
      hasNextPage
      endCursor
    }
    edges {
      node {
        ... on Repository {
          url
          id
          stargazerCount
          forkCount
          description
          issues {
            totalCount
          }
            languages(first: 20) {
            nodes {
              name
            }
            }
          fundingLinks {
            platform
            url
          }      
          pullRequests {
            totalCount
          }
          updatedAt
          object(expression: "HEAD:README.md") {
            ... on Blob {
              text
            }
          }
          repositoryTopics(first: 20) {
            edges {
              node {
                topic {
                  name
                }
              }
            }
          }
        }
      }
    }
      }
    }
```
        % (query, str(after and F'"{after}"' or "null")))
        


async def search_one(query, after=None):
 
 the graphql query 
 with REST, you get what you get. with grapql, you get what you ask for. 
 return await client.execute_async(
 
 { 
 search ( query : "%s" , type : REPOSITORY , first : 50 , after : % s ) { 
 repositoryCount 
 wikiCount 
 pageInfo { 
 hasNextPage 
 endCursor 
 } 
 edges { 
 node { 
 ... on Repository { 
 url 
 id 
 stargazerCount 
 forkCount 
 description 
 issues { 
 totalCount 
 } 
 languages ( first : 20 ) { 
 nodes { 
 name 
 } 
 } 
 fundingLinks { 
 platform 
 url 
 } 
 pullRequests { 
 totalCount 
 } 
 updatedAt 
 object ( expression : "HEAD:README.md" ) { 
 ... on Blob { 
 text 
 } 
 } 
 repositoryTopics ( first : 20 ) { 
 edges { 
 node { 
 topic { 
 name 
 } 
 } 
 } 
 } 
 } 
 } 
 } 
 } 
 } 
 
 
 % (query, str(after and F'"{after}"' or "null")))

In [42]:
%%
    async def search(query, stop=20):
[paginate through a search results](https://til.simonwillison.net/github/graphql-pagination-python) and collect the results.
        
        queries = [await search_one(query)]
        ct = 1 
        while queries[-1]["data"]["search"]:
            if info := queries[-1]["data"]["search"].get("pageInfo"):
                if info["hasNextPage"]:
                    print(queries[-1]["data"]["search"]["pageInfo"]["endCursor"])
                    queries.append(await search_one(query, queries[-1]["data"]["search"]["pageInfo"]["endCursor"]))
                    print(F"done {ct}")
                    ct += 1
                    if ct == stop: break
                    continue
            break
        return queries

async def search(query, stop=20):
 
 paginate through a search results and collect the results. 
 queries = [await search_one(query)]
 ct = 1 
 while queries[-1]["data"]["search"]:
 if info := queries[-1]["data"]["search"].get("pageInfo"):
 if info["hasNextPage"]:
 print(queries[-1]["data"]["search"]["pageInfo"]["endCursor"])
 queries.append(await search_one(query, queries[-1]["data"]["search"]["pageInfo"]["endCursor"]))
 print(F"done {ct}")
 ct += 1
 if ct == stop: break
 continue
 break
 return queries

In [43]:
df = DataFrame(results := await search("topic:assistive-technology", 1))

Y3Vyc29yOjUw
done 1
Y3Vyc29yOjEwMA==
done 2
Y3Vyc29yOjE1MA==
done 3
Y3Vyc29yOjIwMA==
done 4
Y3Vyc29yOjI1MA==
done 5
Y3Vyc29yOjMwMA==
done 6


https://github.com/a11yhood/research/blob/main/2025-01-14-graphql.ipynb

In [36]:
data = df["data"].series(
)["search"].series(
)["edges"].explode(
).series()["node"].series().set_index("id")
data.pullRequests = data.pullRequests.itemgetter("totalCount")
data.issues = data.issues.itemgetter("totalCount")
data["README"] = data.pop("object").itemgetter("text")

data = data.join(
    data.pop("repositoryTopics").series()["edges"]
    .explode().itemgetter("node").itemgetter("topic").itemgetter("name")
    .groupby(level=0).agg(list).rename("topics")
)

In [40]:
data

,url,stargazerCount,forkCount,description,issues,fundingLinks,pullRequests,updatedAt,README,topics
id,,,,,,,,,,
R_kgDOItjjpA,https://github.com/ai-collection/ai-collection,7833,778,The Generative AI Landscape - A Collection of ...,59,"[{'platform': 'LIBERAPAY', 'url': 'https://lib...",639,2025-02-04T14:22:26Z,"# <img width=""20"" style=""margin-right:6px;"" al...","[artificial-intelligence, collections, ai, ass..."
MDEwOlJlcG9zaXRvcnkyMTgzNjE0OA==,https://github.com/OptiKey/OptiKey,4310,506,OptiKey - Full computer control and speech wit...,503,"[{'platform': 'GITHUB', 'url': 'https://github...",413,2025-02-04T00:19:56Z,# OptiKey\n\nOptiKey is an on-screen keyboard ...,"[eye-tracking, eyetracking, eye-tracker, eyes,..."
MDEwOlJlcG9zaXRvcnkzNTM1NjE3OA==,https://github.com/brunopulis/awesome-a11y,1838,145,A curate list about A11Y,20,"[{'platform': 'GITHUB', 'url': 'https://github...",160,2025-02-01T08:06:28Z,# Awesome Accessibility\n\n[![Awesome](https:/...,"[accessibility, wai-aria, wcag, a11y, awesome-..."
MDEwOlJlcG9zaXRvcnkyMDAyNzUxMjQ=,https://github.com/Stypox/dicio-android,862,78,Dicio assistant app for Android,183,"[{'platform': 'LIBERAPAY', 'url': 'https://lib...",86,2025-02-01T18:56:22Z,# Dicio assistant\n\nDicio is a *free and open...,"[assistant, assistive-technology, personal-ass..."
MDEwOlJlcG9zaXRvcnk4MTU3NTc1OQ==,https://github.com/cboard-org/cboard,667,176,Augmentative and Alternative Communication (AA...,596,"[{'platform': 'OPEN_COLLECTIVE', 'url': 'https...",1200,2025-01-25T01:42:45Z,[![Digital public good](https://github.com/cbo...,"[aac, autism, cerebral-palsy, progressive-web-..."
...,...,...,...,...,...,...,...,...,...,...
R_kgDOKqLtnQ,https://github.com/makersmakingchange/Single-M...,0,0,Guide for modifying commercial message playbac...,0,[],0,2024-03-27T19:04:23Z,# Single Message Playback Switch\nThe Single M...,"[accessibility, assistive-technology, disabili..."
R_kgDOImJzBg,https://github.com/obstino-org/facecontrol,0,0,Face Control Android app,0,[],0,2024-11-05T12:31:52Z,## **Face Control Android app**\n### Face Cont...,"[accessibility, assistive-switch, assistive-te..."
MDEwOlJlcG9zaXRvcnk2ODAxNDA5MA==,https://github.com/scify/Memor-i,0,3,None,0,[],8,2024-12-12T22:47:00Z,\n# Memor-i | Memory Game for the visually imp...,"[assistive-technology, java, javafx, javafx-ap..."


In [52]:
data.topics.explode().value_counts().to_frame("count").T.style.set_caption(
    "the column titles are the tags of the returned search results and their counts are in the cells below"
)

topics,assistive-technology,accessibility,openat,python,disability,artificial-intelligence,arduino,aac,machine-learning,wcag,ai,open-source,assistant,eye-tracking,wai-aria,screen-reader,3d-printing,mouse-emulation,joystick,computer-vision,voice-assistant,a11y,python3,deep-learning,disabilities,speech-recognition,android,communication,open-hardware,text-to-speech,asterics-main,switch-control,human-computer-interaction,hardware,voice-recognition,assistive,raspberry-pi,ios,mouse,hacktoberfest,xbox-controller,voice-control,virtual-keyboard,lipsync,lip-mouse,voice-commands,mouth-mouse,switch,awesome,circuitpython,assistive-robotics,keyboard-emulation,robotics,opencv,special-needs,keyboard,blindness,android-application,signal-processing,face-tracking,ocr-recognition,environmental-control,xbox,embedded-systems,pcb,open-source-hardware,awesome-list,education,dicio-assistant,personal-assistant-framework,speech-to-text,personal-assistant,api,cli,go-baby-go,dicio,vuejs,eye-tracker,speech-to-code,gaming,example,neural-network,demo,voice-programming,xbox-adaptive-controller,ape,switch-access,firmware,visually-impaired,biomechanics,ble,openhardware,ocr,switch-interface,bluetooth,assistive-switch,jamstack,fastspeech2,at-solution,blind,trackpoint,cirque,php,smart-home,vue,llm,remote-control,esp32,laravel,diy,stl,opensource,twitter,chrome-extension,natural-language-processing,virtual,screenreader,nvda,laravel-framework,user-experience,pyttsx3,voice-user-interface,adapted-gaming,visual-impairments,productivity,aria,android-app,swift,javascript,cern-ohl,speech,terminal,3d-printable,cerebral-palsy,autism,augmentative-and-alternative-communication,software-development,non-verbal-communication,car,assistant-chat-bots,communication-aid,tts,assistance,electron,mvc,mediapipe,object-detection,navigation,augmentative-communication,flask,alternative-communication,youtube,mathematics,clean-code,nlp,augmented-reality,human-robot-interaction,java,gpt4,pint,web-rtc,visually-impaired-people,virtual-reality,sensact,smartphone,solid,solid-principles,pocketsphinx,counter-strike-global-offensive,bombtimer,timer,ps5,sony-access-controller,wii-nunchuk,brazilian-portuguese,netcle,leonardo,paper-based,thrustmaster,rest,usb-host-shield,poeditor,optical-character-recognition,xac,eslint,flight-stick,laravel-application,laravel-mix,mysql,morse-code,farmer-problem,farm,crop-yield-prediction,crop-recommendation,crop-prediction,crop-disease-detection,tfjs,openai,ruby-on-rails,reactjs,react-bootstrap,drop-down,repl,openscad,auto-complete,markov,react-native,pytorch,language-assistance,disability-support,voice-assistance,speech-impairment,keyboard-navigation,inclusive-design,focus-management,assistive-tech,xml,java-swing,bci,shitposting,playstation-5,joystick-control,instructions,yolox,easyocr,jarvis-ai,system-programming,system,script,mac-app,macos,applescript,engine,automatic-speech-recognition,nlp-machine-learning,ssh,secure-shell,remote-access-tool,recorder,openai-whisper,commbase,albinism,rasberry-pi,voicerecognition,texttospeech,flask-api,rest-api,resnet,imagenet-dataset,imagenet-classifier,stt,accessiblity,design-system,swiftui,hearing-impaired-people,typing,typing-game,typing-trainer,typing-practice,rehabilitation,talkbox,mpr121,inclusion,alternative-input,napari-plugin,napari,python-3,labelling,automation,scanning,diagator,zoom,meetings,timetable,thumbstick,drone,nunchuck,traslation,usb,usb-hid,wii,joy2mouse,function-calling,gemini,vertex-ai,cranelift,independent-living,nunchuk,phonetics,deafblind,logitech-joystick,thrustmaster-joystick,neurodivergent,javaee,user-centered-design,msapi5,aruco-marker-detection,fpga,mobility-aid,obstacle-avoidance,obstacle-detection,sensor-integration,smart-devices,vhdl,pointing-device,fall-detection,localization,memory-augmentation,wordpress-plugin,hci,magnetometer,open-source-project,tesseract,tremor,ipa,eeg-signals-processing,neurosky-mindwave-headset,windows-10,eyetracking,game,hacktoberfest-accepted,supportive,switch-adapted-toy,as